<a href="https://colab.research.google.com/github/hyun-hyang/ResumEmphasizer/blob/main/jd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 4.4 MB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 52.9 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spacy
from spacy.tokens import DocBin
import json

In [3]:
spacy.__version__

'3.4.1'

In [4]:
!nvidia-smi

Tue Aug  9 16:20:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    34W / 250W |    407MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!git clone 'https://github.com/hyun-hyang/ResumEmphasizer.git'

Cloning into 'ResumEmphasizer'...
remote: Enumerating objects: 7485, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 7485 (delta 35), reused 95 (delta 23), pack-reused 7368
Receiving objects: 100% (7485/7485), 38.00 MiB | 22.02 MiB/s, done.
Resolving deltas: 100% (805/805), done.


In [6]:
%cd /content/ResumEmphasizer
!git pull

/content/ResumEmphasizer
Already up to date.


In [7]:
cv_data = json.load(open('/content/ResumEmphasizer/Data_annoctated/mergerd jd.json','r'))

In [8]:
len(cv_data)

159

In [9]:
!python -m spacy init fill-config /content/ResumEmphasizer/Data/base_config.cfg /content/ResumEmphasizer/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/ResumEmphasizer/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
cv_data[0]

{'id': 19,
 'label': [[0, 5, 'company'],
  [337, 357, 'project'],
  [362, 418, 'project'],
  [421, 459, 'job'],
  [475, 481, 'required skills'],
  [483, 488, 'required skills'],
  [489, 500, 'required skills'],
  [505, 513, 'required skills'],
  [642, 690, 'project'],
  [695, 772, 'project'],
  [809, 847, 'benefit'],
  [887, 927, 'benefit'],
  [989, 1057, 'benefit'],
  [1160, 1218, 'benefit'],
  [1464, 1532, 'benefit']],
 'text': "Yapz is a cutting edge, growth-stage startup that allows you to find, connect, and co-host events with clubs in your local area. In addition to their schooling, social, family and work lives, college students partake in extracurricular clubs on campus. To allocate enough time to make those clubs successful is extremely difficult. Yapz makes running a club and finding partner clubs for events easy and time efficient.\n\nWe are looking for a Backend Developer who works with Python, flask, SQLAlchemy and Postgres. The developer should also have a practical under

In [11]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [12]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [13]:
len(train), len(test)

(111, 48)

In [14]:
file=open('error.txt','w')

db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [15]:
len(db.tokens)

48

In [20]:
!python -m spacy train /content/ResumEmphasizer/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-09 16:23:34,760] [INFO] Set up nlp object from config
[2022-08-09 16:23:34,770] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-09 16:23:34,774] [INFO] Created vocabulary
[2022-08-09 16:23:34,775] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a 

In [21]:
nlp = spacy.load('/content/ResumEmphasizer/output/model-best')

In [22]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached PyMuPDF-1.20.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.8 MB)


In [23]:
import sys, fitz

In [25]:
for x in range(1,11):
  fname='/content/ResumEmphasizer/test_jd/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('jd num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')


jd num 1 test
Django Developer  ->>>>>>  job
Self managing, comfortable with remote working Desire to learn  ->>>>>>  required skills
1+ years of Python software development  ->>>>>>  required year
2+ years in any modern language  ->>>>>>  required year
XML  ->>>>>>  required skills
REST  ->>>>>>  required skills
Software architecture patterns  ->>>>>>  required skills
Web Frameworks  ->>>>>>  required skills
GIT version control  ->>>>>>  required skills
MySQL Database Message based  ->>>>>>  required skills
Frontend skills  ->>>>>>  required skills
HTML  ->>>>>>  required skills
CSS  ->>>>>>  required skills
Javascript  ->>>>>>  required skills
JQuery  ->>>>>>  required skills
B.Sc - Computers, Electronics PG: Any Postgraduate - Any Specialization Doctorate: Any Doctorate - Any Specialization"  ->>>>>>  required skills


jd num 2 test
DevOps Engineer  ->>>>>>  job
Kubernetes complete hands-on (3-5 yrs min  ->>>>>>  preferred skills
Docker  ->>>>>>  preferred skills
AWS  ->>>>>>  prefe